In [1]:
import os
import re
import math
import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

cwd = os.getcwd()

In [2]:
sentence_new_parsed = pd.read_csv(os.path.join(cwd, 'data', 'raw', 'Conf_transcripts_new_parsed.csv'))
sentence_new_parsed

,Unnamed: 0,ans_name,answer,ask_name,path,question,time_text,time_text_len,ticker
0,question_1,"george paleologou, will kalutycz","['Yes, generally speaking, a lot of the rebala...",sabahat khan,/backup_disk/conference_calls/PRBZF/4073438-pr...,['Thanks and good afternoon. Can you maybe tal...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF
1,question_2,will kalutycz,"['Yes, it’s kind of a mixture, the Sandwich pl...",sabahat khan,/backup_disk/conference_calls/PRBZF/4073438-pr...,"['Alright, thanks. And then you mentioned that...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF
2,question_3,will kalutycz,"['Yes, again, I mean we deal in all commoditie...",sabahat khan,/backup_disk/conference_calls/PRBZF/4073438-pr...,['Great. And then just if you could give us up...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF
3,question_5,george paleologou,['Thanks George.'],george doumet,/backup_disk/conference_calls/PRBZF/4073438-pr...,"['Hey, good afternoon guys and congrats on the...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF
4,question_6,george paleologou,"['Yes, actually, it was more -- it was not too...",george doumet,/backup_disk/conference_calls/PRBZF/4073438-pr...,['Just looking at Alberta first year-over-year...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF
...,...,...,...,...,...,...,...,...,...
2369695,question_12,adolfo castro,"['Okay. As I said before, today we do not have...",augusto ensiki - morgan stanley,/backup_disk/conference_calls/ASR/1362001-grup...,"['Hi Adolfo, good morning, just a question on ...","Grupo Aeroportuario del Sureste, S.A.B. de C.V...",105,ASR
2369696,question_13,adolfo castro,"['Exactly in the investment, yeah.']",augusto ensiki - morgan stanley,/backup_disk/conference_calls/ASR/1362001-grup...,"['Okay. So then in the, the way that it appear...","Grupo Aeroportuario del Sureste, S.A.B. de C.V...",105,ASR
2369697,question_14,adolfo castro,['You’re welcome.'],augusto ensiki - morgan stanley,/backup_disk/conference_calls/ASR/1362001-grup...,"['Okay perfect. Thank you very much, Adolfo.']","Grupo Aeroportuario del Sureste, S.A.B. de C.V...",105,ASR
2369698,question_15,adolfo castro,"['Yes and getting closer, I am glad, probably ...",adolfo ramos - franklin templeton,/backup_disk/conference_calls/ASR/1362001-grup...,"['Hey Adolfo, good morning everyone. I just wa...","Grupo Aeroportuario del Sureste, S.A.B. de C.V...",105,ASR


### Rename columns & drop cols

In [3]:
sentence_new_parsed.drop(inplace=True, columns=['path', 'time_text_len'])
sentence_new_parsed.columns = ['question_index', 'answeree', 'answer', 'questioner', 'question', 'time_text', 'equity']
sentence_new_parsed.head()

,question_index,answeree,answer,questioner,question,time_text,equity
0,question_1,"george paleologou, will kalutycz","['Yes, generally speaking, a lot of the rebala...",sabahat khan,['Thanks and good afternoon. Can you maybe tal...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF
1,question_2,will kalutycz,"['Yes, it’s kind of a mixture, the Sandwich pl...",sabahat khan,"['Alright, thanks. And then you mentioned that...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF
2,question_3,will kalutycz,"['Yes, again, I mean we deal in all commoditie...",sabahat khan,['Great. And then just if you could give us up...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF
3,question_5,george paleologou,['Thanks George.'],george doumet,"['Hey, good afternoon guys and congrats on the...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF
4,question_6,george paleologou,"['Yes, actually, it was more -- it was not too...",george doumet,['Just looking at Alberta first year-over-year...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF


### Convert answer & question to string

In [4]:
def parse_question_answer(text):
    return text.replace("['", '').replace("']", '').replace('["', '').replace('"]', '')

In [5]:
sentence_new_parsed['question'] = sentence_new_parsed['question'].apply(parse_question_answer)
sentence_new_parsed['answer'] = sentence_new_parsed['answer'].apply(parse_question_answer)

### Parse Time Text

In [6]:
def matching_quater(text):
    try:
        if math.isnan(text):
            return 'NaN'
    except TypeError:
        # match pattern: Q[\d][whitespace]
        matching_result = re.findall(r'Q\d ', text)
        if len(matching_result) != 0:
            return 'Not Parsed' if len(matching_result) != 1 else matching_result[0].strip()
        # matching pattern: (Qtr End 12/31/08)
        matching_result = re.findall(r'Qtr End \d\d\/\d\d\/\d\d', text)
        return 'Not Parsed' if len(matching_result) != 1 else matching_result[0]

def chkList(lst):
    return len(set(lst)) == 1

def matching_year(text):
    try:
        if math.isnan(text):
            return 'NaN'
    except TypeError:
        # match pattern: 2021
        matching_result = re.findall(r'20\d\d', text)
        return matching_result[0] if len(matching_result) != 0 else 'Not Parsed'
#! question here:
#! E.g., Grupo Aeroportuario del Sureste, S.A.B. de C.V. (NYSE:ASR)Q4 2010 Earnings Conference CallFebruary 24, 2011 10:00 AM EST
#! keep first


In [7]:
sentence_new_parsed['quater'] = sentence_new_parsed['time_text'].apply(matching_quater)
sentence_new_parsed['year'] = sentence_new_parsed['time_text'].apply(matching_year) 
sentence_new_parsed.head()

,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
0,question_1,"george paleologou, will kalutycz","Yes, generally speaking, a lot of the rebalanc...",sabahat khan,Thanks and good afternoon. Can you maybe talk ...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
1,question_2,will kalutycz,"Yes, it’s kind of a mixture, the Sandwich plan...",sabahat khan,"Alright, thanks. And then you mentioned that H...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
2,question_3,will kalutycz,"Yes, again, I mean we deal in all commodities ...",sabahat khan,Great. And then just if you could give us upda...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
3,question_5,george paleologou,Thanks George.,george doumet,"Hey, good afternoon guys and congrats on the q...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
4,question_6,george paleologou,"Yes, actually, it was more -- it was not too m...",george doumet,Just looking at Alberta first year-over-year g...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017


## Process Question Index

### Convert to int

In [8]:
sentence_new_parsed['question_index'] = sentence_new_parsed['question_index'].apply(lambda x: x.split('_')[1])
sentence_new_parsed.head()

,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
0,1,"george paleologou, will kalutycz","Yes, generally speaking, a lot of the rebalanc...",sabahat khan,Thanks and good afternoon. Can you maybe talk ...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
1,2,will kalutycz,"Yes, it’s kind of a mixture, the Sandwich plan...",sabahat khan,"Alright, thanks. And then you mentioned that H...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
2,3,will kalutycz,"Yes, again, I mean we deal in all commodities ...",sabahat khan,Great. And then just if you could give us upda...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
3,5,george paleologou,Thanks George.,george doumet,"Hey, good afternoon guys and congrats on the q...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
4,6,george paleologou,"Yes, actually, it was more -- it was not too m...",george doumet,Just looking at Alberta first year-over-year g...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017


### Save to processed

In [9]:
sentence_new_parsed.columns = ['question_index', 'answeree', 'answer', 'questioner', 'question', 'time_text', 'equity', 'quater', 'year']
sentence_new_parsed.to_csv(os.path.join(cwd, 'data', 'processed', 'Conf_transcripts_new_parsed_processed.csv'), index=False)